In [19]:
import os
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow'

In [20]:
os.chdir("../")
%pwd

'c:\\Users\\Lenovo\\Desktop'

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    n_estimators: int  # Add RandomForest specific parameter
    max_depth: int     # Add RandomForest specific parameter


In [3]:
from stb_pfe_mlflow.constants import *
from stb_pfe_mlflow.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.randomforestclassifier  # Update to RandomForest parameters
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,
            n_estimators=params.n_estimators,  # RandomForest parameter
            max_depth=params.max_depth  # RandomForest parameter
        )

        return model_trainer_config


In [5]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load the data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Encode the target column
        label_encoder = LabelEncoder()
        train_y_encoded = label_encoder.fit_transform(train_y)
        test_y_encoded = label_encoder.transform(test_y)

        # Initialize and train RandomForest model
        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth
        )
        rf.fit(train_x, train_y_encoded)

        # Save the model
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

        print("Model training complete and saved.")


In [6]:
try:
    # Step 1: Initialize Configuration Manager
    config = ConfigurationManager()

    # Step 2: Get Model Trainer Configuration
    model_trainer_config = config.get_model_trainer_config()

    # Step 3: Initialize Model Trainer with the configuration
    model_trainer = ModelTrainer(config=model_trainer_config)

    # Step 4: Train the model
    model_trainer.train()

except Exception as e:
    raise e


FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [14]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    random_state: int  # Add this line
    target_column: str

In [15]:
def get_model_trainer_config(self) -> ModelTrainerConfig:
    config = self.config.model_trainer
    params = self.params.randomforestclassifier  # Update this to match your configuration
    schema = self.schema.TARGET_COLUMN

    create_directories([config.root_dir])

    model_trainer_config = ModelTrainerConfig(
        root_dir=config.root_dir,
        train_data_path=config.train_data_path,
        test_data_path=config.test_data_path,
        model_name=config.model_name,
        n_estimators=params.n_estimators,
        max_depth=params.max_depth,
        random_state=params.random_state,  # Add this line
        target_column=schema.name
    )

    return model_trainer_config


In [16]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    random_state: int  # Ensure this is included
    target_column: str

# In your ConfigurationManager
def get_model_trainer_config(self) -> ModelTrainerConfig:
    config = self.config.model_trainer
    params = self.params.randomforestclassifier  # Ensure it points to the correct section
    schema = self.schema.TARGET_COLUMN

    create_directories([config.root_dir])

    model_trainer_config = ModelTrainerConfig(
        root_dir=config.root_dir,
        train_data_path=config.train_data_path,
        test_data_path=config.test_data_path,
        model_name=config.model_name,
        n_estimators=params.n_estimators,
        max_depth=params.max_depth,
        random_state=params.random_state,  # Ensure this line is included
        target_column=schema.name
    )

    return model_trainer_config


In [17]:
import pandas as pd
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load the data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Encode the target column
        label_encoder = LabelEncoder()
        train_y_encoded = label_encoder.fit_transform(train_y)
        test_y_encoded = label_encoder.transform(test_y)

        # Initialize and train Random Forest model
        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            random_state=self.config.random_state
        )
        rf.fit(train_x, train_y_encoded)

        # Save the model
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

        # Make predictions
        predictions = rf.predict(test_x)

        # Calculate accuracy
        accuracy = accuracy_score(test_y_encoded, predictions)
        print(f"Model training complete and saved.")
        print(f"Accuracy of the model: {accuracy * 100:.2f}%")

        return accuracy


In [18]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config() 
    model_trainer = ModelTrainer(config=model_trainer_config)
    accuracy = model_trainer.train() 
except Exception as e:
    logger.error(f"An error occurred during model training: {e}")
    raise e

[2024-09-23 19:01:53,886: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-23 19:01:53,888: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-23 19:01:53,892: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-23 19:01:53,893: INFO: common: created directory at: artifacts]
[2024-09-23 19:01:53,894: INFO: common: created directory at: artifacts/model_trainer]
[2024-09-23 19:01:53,895: ERROR: 376976292: An error occurred during model training: ModelTrainerConfig.__init__() missing 1 required positional argument: 'random_state']


TypeError: ModelTrainerConfig.__init__() missing 1 required positional argument: 'random_state'

In [27]:
print(  # Make predictions
        predictions = rf.predict(test_x))

NameError: name 'rf' is not defined